In [ ]:
from utils import *

In [ ]:
# Parámetros del modelo PDI
alpha = 0.05
lr = 0.001
T_burnin = 5
Csat = 2.0
KI = 1.0
ahead = 1
seasonal_period = 7

In [ ]:
# Simular datos
n_series = 3
n_points = 60
np.random.seed(42)
dates = pd.date_range(start="2023-01-01", periods=n_points, freq="D")

data = []
for i in range(1, n_series + 1):
    key = f"SERIE_{i}"
    y_real = np.random.normal(100, 10, n_points)
    y_pred = y_real + np.random.normal(0, 10, n_points)
    future = [0] * int(n_points * 0.7) + [1] * int(n_points * 0.3)
    data.extend({
        "KEY": key,
        "FECHA": date,
        "Y": real if not fut else np.nan,
        "YHATFIN": pred,
        "FUTURE": fut,
        "YHAT_L": np.nan,
        "YHAT_U": np.nan,
    } for date, real, pred, fut in zip(dates, y_real, y_pred, future))

df = pd.DataFrame(data)

# Aplicar intervalos
df = assign_data_sets(df, date_col="FECHA", future_col="FUTURE", calib_ratio=0.3)
df_pdi = apply_pdi_with_calibration(
    df=df,
    key_col="KEY",
    date_col="FECHA",
    value_col="Y",
    pred_col="YHATFIN",
    lower_col="YHAT_L",
    upper_col="YHAT_U",
    alpha=alpha,
    lr=lr,
    T_burnin=T_burnin,
    Csat=Csat,
    KI=KI,
    ahead=ahead,
    seasonal_period=seasonal_period,
    set_col="SET"
)



In [ ]:
# Visualizar resultados para una serie
plot_series_results_with_sets(df_pdi, key="SERIE_1")

# Visualizar resultados para una serie
plot_series_results_with_sets(df_pdi, key="SERIE_2")

# Visualizar resultados para una serie
plot_series_results_with_sets(df_pdi, key="SERIE_3")

In [ ]:
print(df_pdi.to_string())

In [ ]:
# Dividir el DataFrame en TRAIN, CALIBRATION y TEST
train_df = df_pdi[df_pdi["SET"] == "TRAIN"]
calib_df = df_pdi[df_pdi["SET"] == "CALIBRATION"]
test_df = df_pdi[df_pdi["SET"] == "TEST"]

# Calcular métricas por serie (KEY) para el conjunto de calibración
calib_metrics_by_key = calculate_metrics(
    calib_df,
    value_col="Y",
    lower_col="YHAT_L",
    upper_col="YHAT_U",
    condition_col="KEY"
)

# Mostrar resultados
print("Métricas por KEY (Calibración):")
print(calib_metrics_by_key)